In [ ]:
# A single lightweight component we register in realtime

In [ ]:
from kfp import dsl
import os

@dsl.component
def say_hello(name: str) -> str:
    hello_text = f'Hello, {name}!'
    print(hello_text)
    return hello_text

@dsl.pipeline
def hello_pipeline(recipient: str) -> str:
    hello_task = say_hello(name=recipient)
    return hello_task.output


In [ ]:
from kfp import compiler

compiler.Compiler().compile(hello_pipeline, 'pipeline.yaml')


In [ ]:
# The below SHOULD work but it wasnt on my cluster...feel free to PR the right way to do this..

In [ ]:
from kfp.client import Client

client = Client(host='http://ml-pipeline-ui.kubeflow.svc.cluster.local/pipline')
run = client.create_run_from_pipeline_package(
    'pipeline.yaml',
    arguments={
        'recipient': 'World',
    },
)


In [ ]:
token_file = "/var/run/secrets/kubeflow/pipelines/token"
print(token_file)
with open(token_file) as f:
    token = f.readline()
client = Client(host='http://ml-pipeline-ui.kubeflow/pipeline',
               existing_token=token)
run = client.create_run_from_pipeline_package(
    'pipeline.yaml',
    arguments={
        'recipient': 'World',
    },
)

In [ ]:
# A multiple step pipeline showing passing of things 

In [19]:
from kfp import dsl
import os

@dsl.component
def say_hello(name: str) -> str:
    hello_text = f'Hello, {name}!'
    print(hello_text)
    return hello_text

# Define our response component 
@dsl.component 
def greeting_response(greeting: str) -> str:
    response_text = f'Did I just hear you say {greeting}? Who are you and how do you know my name!?'
    print(response_text)
    return response_text
# Ensure that our response task takes the output from the previous task.
@dsl.pipeline
def hello_pipeline(recipient: str) -> str:
    hello_task = say_hello(name=recipient)
    response_task = greeting_response(greeting=hello_task.output)
    return response_task.output


In [ ]:
from kfp import compiler

compiler.Compiler().compile(hello_pipeline, 'multi_pipeline.yaml')

In [ ]:
# Simple Math Example

In [26]:
from kfp import dsl

@dsl.component
def square(x: float) -> float:
    return x ** 2

@dsl.component
def add(x: float, y: float) -> float:
    return x + y

@dsl.component
def square_root(x: float) -> float:
    return x ** .5

@dsl.pipeline
def pythagorean(a: float, b: float) -> float:
    a_sq_task = square(x=a)
    b_sq_task = square(x=b)
    sum_task = add(x=a_sq_task.output, y=b_sq_task.output)
    return square_root(x=sum_task.output).output

In [27]:
from kfp import compiler
compiler.Compiler().compile(pythagorean, 'math_pipeline.yaml')
